In [0]:
import pandas as pd

train = pd.read_csv('drive/My Drive/Colab Notebooks/NLP/train.tsv', sep="\t")

In [3]:
train.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [4]:
train.groupby("Sentiment").count()

,PhraseId,SentenceId,Phrase
Sentiment,,,
0,7072,7072,7072
1,27273,27273,27273
2,79582,79582,79582
3,32927,32927,32927
4,9206,9206,9206


In [0]:
train["Sentiment"].replace(0, value = "negatif" , inplace = True)
train["Sentiment"].replace(1, value = "negatif" , inplace = True)

train["Sentiment"].replace(3, value = "pozitif" , inplace = True)
train["Sentiment"].replace(4, value = "pozitif" , inplace = True)

In [0]:
train = train[(train.Sentiment == "negatif") | (train.Sentiment == "pozitif")]

In [0]:
df = pd.DataFrame()
df["Phrase"] = train["Phrase"]
df["sonuç"] = train["Sentiment"]

In [8]:
df.head()

,Phrase,sonuç
0,A series of escapades demonstrating the adage ...,negatif
21,good for the goose,pozitif
22,good,pozitif
33,"the gander , some of which occasionally amuses...",negatif
46,amuses,pozitif


In [0]:
#büyük - küçük dönüşümü 
df['Phrase'] = df['Phrase'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#noktalama işaretleri kaldırılması
df['Phrase'] = df['Phrase'].str.replace('[^\w\s]','')

#sayıların kaldırılması
df['Phrase'] = df['Phrase'].str.replace('\d','')


In [10]:
df.head()

,Phrase,sonuç
0,a series of escapades demonstrating the adage ...,negatif
21,good for the goose,pozitif
22,good,pozitif
33,the gander some of which occasionally amuses ...,negatif
46,amuses,pozitif


In [0]:
#stopwords bulunması
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')
df['Phrase'] = df['Phrase'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

In [12]:
df.head()

,Phrase,sonuç
0,series escapades demonstrating adage good goos...,negatif
21,good goose,pozitif
22,good,pozitif
33,gander occasionally amuses none amounts much s...,negatif
46,amuses,pozitif


In [0]:
#seyreklerin dataların silinmesi silinmesi
sil = pd.Series(' '.join(df['Phrase']).split()).value_counts()[-1000:]
df['Phrase'] = df['Phrase'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))

In [14]:
#lemmatize
from textblob import Word
nltk.download('wordnet')
df['Phrase'] = df['Phrase'].apply(lambda x: " ".join([Word(i).lemmatize() for i in x.split()])) 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
df.head()

,Phrase,sonuç
0,series demonstrating adage good goose also goo...,negatif
21,good goose,pozitif
22,good,pozitif
33,gander occasionally amuses none amount much story,negatif
46,amuses,pozitif


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76478 entries, 0 to 156057
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Phrase  76478 non-null  object
 1   sonuç   76478 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


In [0]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
df["sonuç"] = encoder.fit_transform(df["sonuç"])

In [18]:
df.head()

,Phrase,sonuç
0,series demonstrating adage good goose also goo...,0
21,good goose,1
22,good,1
33,gander occasionally amuses none amount much story,0
46,amuses,1


In [0]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(df["Phrase"],df["sonuç"],  random_state = 1)

**Count Vektörleri**

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(train_x)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
x_train_count = vec.transform(train_x)
x_test_count = vec.transform(test_x)

In [22]:
x_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# TF-IDF

In [0]:
#wordlevel

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

In [0]:
# ngram level tf-idf

In [26]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range = (2,3))
tf_idf_ngram_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(train_x)
x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

**Chars**

In [29]:
tf_idf_chars_vectorizer = TfidfVectorizer(analyzer = "char", ngram_range = (2,3))
tf_idf_chars_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
x_train_tf_idf_chars = tf_idf_chars_vectorizer.transform(train_x)
x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

**Modeller**

In [42]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
nb = MultinomialNB()
nb_model = nb.fit(x_train_count,train_y)
accuracy = cross_val_score(nb_model, 
                                 x_test_count, 
                                 test_y, 
                                 cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8326882845188285


In [30]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
nb = MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_word,train_y)
accuracy = cross_val_score(nb_model, 
                                 x_test_tf_idf_word, 
                                 test_y, 
                                 cv = 10).mean()

print("TF-Idf Word Level Doğruluk Oranı:", accuracy)

TF-Idf Word Level Doğruluk Oranı: 0.8343096234309624


In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
nb = MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_ngram,train_y)
accuracy = cross_val_score(nb_model, 
                                 x_test_tf_idf_ngram, 
                                 test_y, 
                                 cv = 10).mean()

print("TF-Idf N-Gram Doğruluk Oranı:", accuracy)

TF-Idf N-Gram Doğruluk Oranı: 0.7686192468619246


In [34]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
nb = MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_chars,train_y)
accuracy = cross_val_score(nb_model, 
                                 x_test_tf_idf_chars, 
                                 test_y, 
                                 cv = 10).mean()

print("TF-Idf Chars Doğruluk Oranı:", accuracy)

TF-Idf Chars Doğruluk Oranı: 0.7561192468619248


In [43]:
import xgboost
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_count,train_y)
accuracy = cross_val_score(xgb_model, 
                                   x_test_count, 
                                   test_y, 
                                   cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.6336297071129706


In [37]:
import xgboost
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_word,train_y)
accuracy = cross_val_score(xgb_model, 
                                   x_test_tf_idf_word, 
                                   test_y, 
                                   cv = 10).mean()

print("Tf-İdf Word Level Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.634152719665272
